<a href="https://colab.research.google.com/github/LetsTrie/COVID-19-BD-Data/blob/master/test_rates/TestRateClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.signal import argrelextrema
from sklearn.linear_model import Ridge
from sklearn.neighbors.kde import KernelDensity
from pandas import DataFrame
from numpy import *
import copy
import json, io
import sys
import math
import random
from copy import deepcopy
from sklearn.cluster import KMeans

In [187]:
main_data = pd.read_csv('modified_data.csv')
main_data_frame = main_data.copy()
main_data_frame = main_data_frame.values
bd_data = main_data_frame[4]

In [188]:
TEST_RATE_COLUMN = 2
cluster_sizes = [0, 0, 0, 6, 5, 6, 5, 4]

def is_null(v): 
    return not v or np.isnan(v)

In [189]:
# Points for clustering
def make_points_for_cluster(param_index): 
    dataset = []
    for i in range(len(main_data_frame)):
        if (not is_null(main_data_frame[i][param_index])): 
            dataset.append((main_data_frame[i][param_index], 0))
    
    return dataset

def form_single_param_array(param_index):
    dataset = []
    for i in range(len(main_data_frame)):
        if (not is_null(main_data_frame[i][param_index])): 
            dataset.append(main_data_frame[i][param_index])

    return dataset

# def test_cluster_size(param_index, a=3, b=20):
#   data = make_points_for_cluster(param_index)
#   y = []
#   x = range(a,b)
#   for i in range(a, b):
#     model = KMeans(n_clusters=i)
#     data = np.array(make_points_for_cluster(param_index))
#     model.fit(data)
#     y.append(model.inertia_)
#   plt.plot(x, y)
#   plt.show()
    
# test_cluster_size(6)

def getSortedFrame(param_index): 
    sortedFrame = []
    for i in range(len(main_data_frame)):
        if(not is_null(main_data_frame[i][param_index])): 
            sortedFrame.append(main_data_frame[i])
        
    return sorted(sortedFrame, key=lambda x:x[param_index],reverse=True)

def getSortedCluster(cluster): 
    sortedCluster = []
    for value in cluster:
        sortedCluster.append(value[0])
    
    return sorted(sortedCluster, reverse=True)

def refine(cluster, param_index):
    _cluster = []
    sortedFrame = getSortedFrame(param_index)
    sortedCluster = getSortedCluster(cluster)
    
    print(f'For Parameter {param_index}')
    print(f'Value Starts with {cluster[len(cluster) - 1]} and ends with {cluster[0]}')
    cluster_index = 0
    for i in range(len(sortedFrame)):
        if cluster_index < len(sortedCluster):
            if sortedFrame[i][param_index] == sortedCluster[cluster_index]: 
                cluster_index += 1
                if sortedFrame[i][1] == 'Bangladesh': 
                    continue;
                _cluster.append(sortedFrame[i])
    
    return _cluster

def form_dataset_kmeans(param_index, k): 
    param_array = make_points_for_cluster(param_index)
    model = KMeans(n_clusters=k)
    model.fit(param_array)
    clusters = []

    for i in range(len(model.labels_)):
        clusters.append([])
    for i in range(len(model.labels_)):
        clusters[model.labels_[i]].append(param_array[i])
    for cluster in clusters:
        if (bd_data[param_index], 0) in cluster:
            return refine(cluster, param_index)

def weighted_mean(data, bd_data, param_index): 
    weights = []
    avg = 0
    
    for i in range(len(data)):
        weight = 1
        if bd_data - data[i][param_index] != 0:
            weight = 1 / abs(bd_data - data[i][param_index])
        weights.append(weight)
    
    _max = np.max(weights)
    norm_weights = list(map(lambda x: x/_max, weights)) 
    
    for i in range(len(data)):
        avg += (data[i][TEST_RATE_COLUMN]*norm_weights[i])
    
    return avg / np.sum(norm_weights)


def correlation(set1, set2):
    mean1, mean2 = np.mean(set1), np.mean(set2)
    numer, denom = 0, 0

    for i in range(len(set1)):
        numer += (set1[i] - mean1) * (set2[i] - mean2)
        denom += (set1[i] - mean1) * (set1[i] - mean1) * (set2[i] - mean2) * (set2[i] - mean2)

    return abs(numer / math.sqrt(denom))

In [190]:
bd_avg_test_rate = bd_data[2]

FROM_GDP_INDEX = 3
UPTO_END_INDEX = 8

def compute():
    param_weight_sum = 0
    test_rate = 0

    for i in range(FROM_GDP_INDEX, UPTO_END_INDEX):
        dataset = form_dataset_kmeans(i, cluster_sizes[i])
        avg = weighted_mean(dataset, bd_data[i], i)
        weight = correlation(form_single_param_array(i), form_single_param_array(TEST_RATE_COLUMN))
        test_rate += (weight * avg)
        param_weight_sum += weight

        print('Cluster Size For Parameter', i, ':', len(dataset))
        print('Avg Test Per Million For Parameter', i, ':', avg)
        print('---------------------------\n')

    test_rate = test_rate / param_weight_sum
    return (test_rate, test_rate / bd_avg_test_rate)

In [191]:
res = compute()
print(res)
print(f"BD average test per million: {bd_avg_test_rate}")

For Parameter 3
Value Starts with (1899.775, 0) and ends with (3523.9840000000004, 0)
Cluster Size For Parameter 3 : 34
Avg Test Per Million For Parameter 3 : 106.01130294245947
---------------------------

For Parameter 4
Value Starts with (97338583, 0) and ends with (164689383, 0)
Cluster Size For Parameter 4 : 9
Avg Test Per Million For Parameter 4 : 662.413633323683
---------------------------

For Parameter 5
Value Starts with (1454.037, 0) and ends with (1265.036, 0)
Cluster Size For Parameter 5 : 2
Avg Test Per Million For Parameter 5 : 2574.9327601796786
---------------------------

For Parameter 6
Value Starts with (1.7, 0) and ends with (0.8, 0)
Cluster Size For Parameter 6 : 34
Avg Test Per Million For Parameter 6 : 338.2304650862162
---------------------------

For Parameter 7
Value Starts with (2.28715, 0) and ends with (3.41984, 0)
Cluster Size For Parameter 7 : 41
Avg Test Per Million For Parameter 7 : 231.80072884683761
---------------------------

(729.5897835401775, 1